In [22]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Activation, Conv2, Dense, Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub

import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy

from PIL import Image

In [ ]:
# 画像から瞳部分をトリミングして保存する
# TODO: 瞳だけじゃなくて鼻とか口とかもトリミングされるので、何か対策を入れるか、処理後に手動で削除する必要あり


# 顔・瞳検出
# https://note.nkmk.me/python-opencv-face-detection-haar-cascade/
face_cascade_path = 'D:/python/OpenCV/data/haarcascades/haarcascade_frontalface_default.xml'
eye_cascade_path = 'D:/python/OpenCV/data/haarcascades/haarcascade_eye.xml'

face_cascade = cv2.CascadeClassifier(face_cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)


# 瞳だけトリミングした画像を保存する
eye_path = image_path + "/eye"
if not(os.path.exists(eye_path)):
    # もうeyeフォルダあったら処理しない
    # なかったらeyeフォルダ作る
    os.mkdir(eye_path)

    # 画像全部見る
    for file in glob.glob(image_path + "/**/*", recursive=True):
        if os.path.splitext(file)[1] in [".jpg", ".png", ".bmp"]:
            # cv2.imreadが日本語ファイル名に対応していないので、一旦Pillowで読み込んでから変換する
            # https://qiita.com/derodero24/items/f22c22b22451609908ee
            src = np.array(Image.open(file), dtype=np.uint8)
            if src.ndim == 2:  # モノクロ
                pass
            elif src.shape[2] == 3:  # カラー
                src = cv2.cvtColor(src, cv2.COLOR_RGB2BGR)
            elif src.shape[2] == 4:  # 透過
                src = cv2.cvtColor(src, cv2.COLOR_RGBA2BGRA)       
            src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

            faces = face_cascade.detectMultiScale(src_gray)

            # 保存先パス周りの整備(フォルダなかったら作る)
            dst_path = (eye_path + file.replace(image_path,"")).replace(os.sep,"/")
            os.makedirs(os.path.dirname(dst_path), exist_ok=True)
            split_dst_path = os.path.splitext(dst_path)
        
            # 顔→瞳を検出
            for x, y, w, h in faces:
                face = src[y: y + h, x: x + w]
                face_gray = src_gray[y: y + h, x: x + w]
                eyes = eye_cascade.detectMultiScale(face_gray)
                eye_count = 0
                for (ex, ey, ew, eh) in eyes:
                    # 瞳1つずつトリミング
                    eye = src[y + ey: y + ey + eh, x + ex: x + ex + ew]
                
                    # 保存
                    # TODO: (放置でもいいかも)日本語だけは文字化けするので元々のファイル名を維持できない
                    each_dst_path = split_dst_path[0] + "_" + str(eye_count) + split_dst_path[1]
                    cv2.imwrite(each_dst_path, eye)
                
                    eye_count += 1
print("end")

In [ ]:
# 瞳画像をさらにトリミングして範囲を絞る


# フォルダ指定
src_dir = "D:/python/Image/eye"
dst_dir = "D:/python/Image/eye_trim"

# 幅・高さのdiv分の1だけ落とす
div = 5.

# 画像全部見る
for file in glob.glob(src_dir + "/**/*", recursive=True):
    if os.path.splitext(file)[1] in [".jpg", ".png", ".bmp"]:
        # cv2.imreadが日本語ファイル名に対応していないので、一旦Pillowで読み込んでから変換する
        # https://qiita.com/derodero24/items/f22c22b22451609908ee
        src = np.array(Image.open(file), dtype=np.uint8)
        if src.ndim == 2:  # モノクロ
            pass
        elif src.shape[2] == 3:  # カラー
            src = cv2.cvtColor(src, cv2.COLOR_RGB2BGR)
        elif src.shape[2] == 4:  # 透過
            src = cv2.cvtColor(src, cv2.COLOR_RGBA2BGRA)       
        
        # トリミング
        y = int(src.shape[0] / div)
        x = int(src.shape[1] / div)
        dst = src[y: (int(div - 1)) * y, x: (int(div - 1)) * x]
        
        # 保存先パス周りの整備(フォルダなかったら作る)
        dst_path = (dst_dir + file.replace(src_dir,"")).replace(os.sep,"/")
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        
        # 保存
        cv2.imwrite(dst_path, dst)
print("end")